In [1]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
import cv2
from PyQt5 import QtCore
from PyQt5 import QtWidgets

# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

# in case you are in debug mode
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

# use this function to create a Viewer, solves the problem of window not raising
viewer=None
def CreateViewer():
    global viewer
    viewer=PyViewer()
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    viewer_py.setVisible(True)
    viewer_py.show()
    viewer_py.setFocus()
    viewer_py.showMaximized()
    viewer_py.activateWindow()
    return viewer

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets']
Starting OpenVisus C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...
QT5_DIR c:\python38\lib\site-packages\PyQt5\Qt


In [2]:
# do not change this cell for very good reasons (otherwise Qt5 loop won't work)
# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
import time
time.sleep(2)

Define some utilities to perform viewer animations

In [28]:
# //////////////////////////////////////////////////////////////
class RunActions:
    
    def __init__(self):
        self.v=[]
        self.cursor=0
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.onTimer)
        
    def addAction(self, fn,**kwargs):
        self.v.append(lambda : fn(**kwargs))
        
    def addSleep(self,msec):
        self.v.append(msec)
        
    def onTimer(self):
        
        # if the last action was an action and the viewer is still running, I need to wait
        if self.cursor>0 and callable(self.v[self.cursor-1]) and viewer.isRunning():
            return
        
        self.timer.stop()
        
        if self.cursor>= len(self.v):
            return
        
        cur=self.v[self.cursor]
        self.cursor+=1
        
        # is a delay?
        if isinstance(cur,int):
            print("*** Sleep",cur)
            self.timer.start(cur) 
            
        # is a real action, run it and later wait for completition
        elif callable(cur):
            cur()
            self.timer.start(20) 
            
        # there must be a problem
        else:
            raise Exception("internal error")
   
    def start(self): 
        self.timer.start(1) 
        
    def stop(self):
        self.timer.stop()
        
print("Utilities defined")

Utilities defined


Some predefined actions

In [29]:
def RotateScene(axis=(0,0,1),angle=10):
    print("*** RotateScene",axis,angle)
    glcamera=viewer.getGLCamera()
    viewer.getGLCamera().setRotation(glcamera.getRotation() * Quaternion(Point3d(axis),math.radians(angle)))
    viewer.refreshAll()
    viewer.postRedisplay()
    
def TakeSnapshot(filename="temp.png",):
    print("*** TakeSnapshotAction",filename)  
    # viewer.takeSnapshot(False,filename) BROKEN
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    screenshot = QtWidgets.QApplication.primaryScreen().grabWindow(viewer_py.winId() )
    screenshot.save(filename)
    
def OpenScene(filename=""):
    print("*** OpenScene",filename)
    viewer.open(filename)
    
def DropSelection():
    print("*** DropSelection")
    viewer.dropSelection()
    
def HideDatasetBounds(uuid="dataset"):
    print("*** HideDatasetBounds",uuid)
    # **** make sure that the dataset node has UUID `dataset` (you can save an xml and inspect it) ***
    dataset_node=DatasetNode.castFrom(viewer.findNodeByUUID(uuid))
    if dataset_node:
        dataset_node.setShowBounds(False)
    else:
        print("Failed to find a node with uuid",uuid)

In [30]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [31]:
filename=r"D:\visus_demo\scenes\battery.xml"
actions=RunActions()
actions.addAction(OpenScene,filename=filename)
actions.addAction(DropSelection)
actions.addAction(HideDatasetBounds,uuid="dataset")
for I in range(5):
    actions.addAction(RotateScene,axis=(0,0,1),angle=10)
    actions.addAction(TakeSnapshot,filename=os.path.splitext(filename)[0] + ".{:03d}.png".format(I))
    
    # you can add some sleep between actions but it's not necessary
    # automatically it can wait for the viewer to become idle    
    actions.addSleep(50) 
    
actions.start()

*** OpenScene D:\visus_demo\scenes\battery.xml
*** DropSelection
2021-06-01 16:14:24.271742 PyScriptingNode Got in input (128, 128, 256) uint8 origin Volume 
2021-06-01 16:14:24.273774 PyScriptingNode Output is  (128, 128, 256) uint8 msec 2 
*** HideDatasetBounds dataset
*** RotateScene (0, 0, 1) 10
2021-06-01 16:14:24.458396 PyScriptingNode Got in input (128, 128, 256) uint8 origin Volume 
2021-06-01 16:14:24.460421 PyScriptingNode Output is  (128, 128, 256) uint8 msec 1 
*** TakeSnapshotAction D:\visus_demo\scenes\battery.000.png
*** Sleep 50
*** RotateScene (0, 0, 1) 10
*** TakeSnapshotAction D:\visus_demo\scenes\battery.001.png
2021-06-01 16:14:24.899151 PyScriptingNode Got in input (128, 128, 256) uint8 origin Volume 
2021-06-01 16:14:24.901151 PyScriptingNode Output is  (128, 128, 256) uint8 msec 2 
2021-06-01 16:14:24.960731 PyScriptingNode Got in input (128, 256, 256) uint8 origin Volume 
2021-06-01 16:14:24.962752 PyScriptingNode Output is  (128, 256, 256) uint8 msec 2 
*** Sl

In [15]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [32]:
snapshots=sorted(glob.glob(r"D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot*xml"))

# limit for debugging...
snapshots=snapshots[:2] 

actions=RunActions()
for I,snapshot in enumerate(snapshots):
    print("\t",I,snapshot)
    actions.addAction(OpenScene,filename=snapshot)
    actions.addAction(DropSelection)
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(TakeSnapshot,filename=os.path.splitext(snapshot)[0] + ".png")
    
    # you can add some sleep between actions but it's not necessary
    # automatically it can wait for the viewer to become idle
    actions.addSleep(10) 
    
actions.start()

	 0 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531145607684.xml
	 1 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531145641394.xml
	 2 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531145646426.xml
	 3 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531145715830.xml
	 4 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531145847913.xml
	 5 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531145955592.xml
	 6 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150116775.xml
	 7 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150230900.xml
	 8 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150304829.xml
	 9 D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150311433.xml


2021-06-01 16:14:58.120923 PyScriptingNode Got in input (1, 484, 497) float32 origin Slice 
2021-06-01 16:14:58.122937 PyScriptingNode Output is  (1, 484, 497) float32 msec 2 
2021-06-01 16:14:58.123925 PyScriptingNode Got in input (1022, 344, 597) float32 origin Volume 
2021-06-01 16:14:58.124922 PyScriptingNode Output is  (1022, 344, 597) float32 msec 1 
2021-06-01 16:14:58.777921 PyScriptingNode Got in input (1, 968, 995) float32 origin Slice 
2021-06-01 16:14:58.779921 PyScriptingNode Output is  (1, 968, 995) float32 msec 2 
2021-06-01 16:15:00.500934 PyScriptingNode Got in input (1, 1245, 1991) float32 origin Slice 
2021-06-01 16:15:00.502933 PyScriptingNode Output is  (1, 1245, 1991) float32 msec 3 
*** HideDatasetBounds dataset
*** TakeSnapshotAction D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150116775.png
*** Sleep 10
*** OpenScene D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150230900.xml
*** DropSelec

2021-06-01 16:15:15.859406 PyScriptingNode Got in input (1, 1245, 1991) float32 origin Slice 
2021-06-01 16:15:15.861393 PyScriptingNode Output is  (1, 1245, 1991) float32 msec 2 
*** HideDatasetBounds dataset
*** TakeSnapshotAction D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150323769.png
*** Sleep 10
*** OpenScene D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150331744.xml
*** DropSelection
2021-06-01 16:15:16.304960 PyScriptingNode Got in input (1, 121, 124) float32 origin Slice 2021-06-01 16:15:16.306959 PyScriptingNode Got in input (464, 172, 273) float32 origin Volume 
2021-06-01 16:15:16.308973 PyScriptingNode Output is  (464, 172, 273) float32 msec 2 

2021-06-01 16:15:16.309958 PyScriptingNode Output is  (1, 121, 124) float32 msec 5 
2021-06-01 16:15:16.367973 PyScriptingNode Got in input (1, 242, 248) float32 origin Slice 
2021-06-01 16:15:16.369973 PyScriptingNode Output is  (1, 242, 248) float32 msec 

2021-06-01 16:15:30.207223 PyScriptingNode Got in input (982, 344, 574) float32 origin Volume 2021-06-01 16:15:30.209207 PyScriptingNode Got in input (1, 512, 512) float32 origin Slice 
2021-06-01 16:15:30.211212 PyScriptingNode Output is  (982, 344, 574) float32 msec 4 

2021-06-01 16:15:30.211212 PyScriptingNode Output is  (1, 512, 512) float32 msec 2 
2021-06-01 16:15:31.219223 PyScriptingNode Got in input (1, 1024, 1024) float32 origin Slice 
2021-06-01 16:15:31.221209 PyScriptingNode Output is  (1, 1024, 1024) float32 msec 2 
2021-06-01 16:15:32.978221 PyScriptingNode Got in input (1, 1350, 1949) float32 origin Slice 
2021-06-01 16:15:32.980222 PyScriptingNode Output is  (1, 1350, 1949) float32 msec 2 
*** HideDatasetBounds dataset
*** TakeSnapshotAction D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150413960.png
*** Sleep 10
*** OpenScene D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot.20210531150423331.xml
*** DropSel

False
